# Car ride-share potential in mid-size U.S. cities from geographic spread

This notebook supports the IBM Data Science Specialization on Coursera, per official report PDF. For all details, see the PDF

## Extract geographic location of mid-size U.S. cities

The 2017 U.S. census estimate for city size can be obtained from https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk . It is faithfully represented in a corresponding Wikipedia page https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308 (retrieved 3 March 2019) from which it can be easily parsed.

In [1]:
import pandas as pd
import numpy as np

Following https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 (retrieved 24 Feb 2019) using BeautifulSoup to get a parseable representation of the Wikipedia page, then load the table with all cities into `city_table`:

In [43]:
import requests
url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))

<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></span></a></span></small>
</td></tr></tbody></table>


From `city_table` find all cities with an estimated 2017 population between 300,000 and 400,000 and parse out the latitude and longitude into numeric values:
* City name is the second column (remove references in square brackets if present),
* city state is the third column,
* population is the 4th column (remove thousands-separator commas before interpreting as integer),
* lattitude and longitude is contained in the 11th column, but has to be substring-filtered.

In [56]:
import re

l = []

table_rows = My_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']
print(cities_df)

(ignoring empty row)
         City name    City state  Population  Latitude  Longitude
0        Arlington         Texas      396394   32.7007   -97.1247
1      New Orleans     Louisiana      393292   30.0534   -89.9345
2          Wichita        Kansas      390591   37.6907   -97.3459
3        Cleveland          Ohio      385525   41.4785   -81.6794
4            Tampa       Florida      385430   27.9701   -82.4797
5      Bakersfield    California      380874   35.3212  -119.0183
6           Aurora      Colorado      366623   39.6880  -104.6897
7          Anaheim    California      352497   33.8555  -117.7601
8         Honolulu        Hawaii      350395   21.3243  -157.8476
9        Santa Ana    California      334136   33.7363  -117.8830
10       Riverside    California      327728   33.9381  -117.3932
11  Corpus Christi         Texas      325605   27.7543   -97.1734
12       Lexington      Kentucky      321959   38.0407   -84.4583
13        Stockton    California      310496   37.9763 

In [17]:
# persist the DataFrame
cities_df.to_csv('cities_df.csv')

NameError: name 'cities_df' is not defined

## Get venues using Foursquare

(this cell contains the Foursquare API credentials)

<!--
CLIENT_ID = 'NBLOR5JJCSM43LTXYWBQYVJ5U3LMNZ2ULCHERZAZVLJTHBYA' # your Foursquare ID
CLIENT_SECRET = 'D1G4RELNK2MGSOZSO1C4DTGPYBAWHIW0MQJTXWSBTGH2JL41' # your Foursquare Secret
-->

In [30]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


In [31]:
# (optional: restore all of the above data from storaget, import all libraries from above)
import pandas as pd
import numpy as np
import requests
import re

cities_df = pd.read_csv('cities_df.csv')

Define a Foursquare query that gets all venues within a default radius of 500 meters around a latitude and longitude. The number of venues returned is capped at 100 by default.

In [64]:
def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request
    results_raw = requests.get(url)
    results = results_raw.json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    
    return(nearby_venues)

### Test number of venues finder

Using the first record in the cities_df DataFrame, test the function defined above.

In [65]:
print(cities_df.head(1))

   Unnamed: 0  City name City state  Population  Latitude  Longitude
0           0  Arlington      Texas      396394   32.7007   -97.1247


In [66]:
venues_df = getVenuesNearLatLong(cities_df.Latitude[0], cities_df.Longitude[0])

found 7 venues within 500 meters of 32.7007/-97.1247


In [68]:
print('venues_df.shape = {}'.format(venues_df.shape))
venues_df

venues_df.shape = (7, 4)


,Latitude,Longitude,Venue,Venue Category
0,32.7007,-97.1247,Krispy Kreme Doughnuts,Donut Shop
1,32.7007,-97.1247,Kenner's Kolache Bakery,Breakfast Spot
2,32.7007,-97.1247,Texas Vision Care,Optical Shop
3,32.7007,-97.1247,Cooper St Bakery,Bakery
4,32.7007,-97.1247,El Pollo Regio,Mexican Restaurant
5,32.7007,-97.1247,Avis Car Rental,Rental Car Location
6,32.7007,-97.1247,Metro Flex,Gym


### Search hex grid around a given coordinate

Define a function that takes a latitude and longitude and return the venues at 6 coordinate points around that location, in a hex grid. Each point in the hex grid will be labeled by an integer as shown in the following diagram around the origin (0,0):

```
        ( -1, 1 )      ( 0, 1 )
                \      /
                 \    /
( -1, 0 )  ---  ( 0, 0 )  ---  ( 1, 0 )
                  /  \
                 /    \
        ( 0, -1 )     ( 1, -1 )
```

Given a latitude and longitude, the entire hex grid can therefore be described by a set of tuples `( x, y )`. The function will collect the venues result and append it to a dictionary that is keyed by these `( x, y )` tuples.

In [69]:
# TODO continue here